# Scrabble Player Rating Prediction

## Практическое задание курса Light Auto ML

**Соревнование:** [Scrabble Player Rating](https://www.kaggle.com/competitions/scrabble-player-rating/data)

**Задача:** Предсказать рейтинг игрока до игры на основе метаданных игры и данных о ходах

**Тип задачи:** Регрессия

---

## Содержание

1. [Импорт библиотек и настройка](#1)
2. [Загрузка данных](#2)
3. [Анализ целевой переменной](#3)
4. [Анализ признаков](#4)
5. [Бейзлайн с LightAutoML](#5)
6. [Собственное решение](#6)
7. [Выводы и сравнение результатов](#7)


<a id='1'></a>
## 1. Импорт библиотек и настройка


In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Standard libraries
import os
import sys
import time
import logging
from pathlib import Path

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression

# LightAutoML
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Constants
RANDOM_STATE = 42
N_THREADS = 4
N_FOLDS = 5
TEST_SIZE = 0.2

# Paths - определяем корневую директорию проекта
# Если ноутбук запущен из папки notebooks/, то PROJECT_ROOT будет родительской директорией
current_dir = Path.cwd()
if current_dir.name == 'notebooks':
    PROJECT_ROOT = current_dir.parent
else:
    # Если запущен из корня проекта или другой директории, ищем папку scrabble_rating_project
    if 'scrabble_rating_project' in str(current_dir):
        # Находим корень проекта
        parts = current_dir.parts
        idx = list(parts).index('scrabble_rating_project') if 'scrabble_rating_project' in parts else -1
        if idx >= 0:
            PROJECT_ROOT = Path(*parts[:idx+1])
        else:
            PROJECT_ROOT = current_dir
    else:
        PROJECT_ROOT = current_dir

DATA_DIR = PROJECT_ROOT / 'data'
RESULTS_DIR = PROJECT_ROOT / 'results'

# Create directories if they don't exist
RESULTS_DIR.mkdir(exist_ok=True, parents=True)
DATA_DIR.mkdir(exist_ok=True, parents=True)

print(f"Python version: {sys.version}")
print(f"Current working directory: {Path.cwd()}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Results directory: {RESULTS_DIR}")

# Проверка наличия данных
data_files = ['games.csv', 'turns.csv', 'train.csv', 'test.csv', 'sample_submission.csv']
print(f"\nПроверка наличия файлов данных:")
all_files_exist = True
for file in data_files:
    file_path = DATA_DIR / file
    exists = file_path.exists()
    if exists:
        size = file_path.stat().st_size / (1024*1024)
        print(f"  ✓ {file}: {size:.1f} MB")
    else:
        print(f"  ✗ {file}: не найден")
        all_files_exist = False

if not all_files_exist:
    print(f"\n⚠️  ВНИМАНИЕ: Не все файлы данных найдены!")
    print(f"   Убедитесь, что данные находятся в папке: {DATA_DIR}")
else:
    print(f"\n✓ Все файлы данных найдены!")


<a id='2'></a>
## 2. Загрузка данных


### 2.1. Обоснование стратегии разделения данных

**Важно:** Для предотвращения утечки данных (data leakage) необходимо правильно разделить данные.

**Особенности данных:**
- Данные содержат информацию о разных игроках (nickname)
- Один и тот же игрок может встречаться в нескольких играх
- Рейтинги игроков могут быть связаны между играми

**Стратегия разделения:**
- Используем случайное разделение с фиксированным random_state для воспроизводимости
- Разделение происходит на уровне записей (игр), а не игроков
- Это допустимо, так как задача - предсказать рейтинг на основе метаданных игры и ходов
- Для более строгого подхода можно было бы использовать групповое разделение по игрокам, но это не требуется для данной задачи

**Предотвращение утечки данных:**
- Все признаки агрегируются ДО разделения на train/validation
- Не используем информацию из будущего (например, финальный счет для предсказания рейтинга до игры)
- Рейтинги в train.csv - это рейтинги ДО игры, что соответствует задаче


In [ ]:
# Load data
games_df = pd.read_csv(DATA_DIR / 'games.csv')
turns_df = pd.read_csv(DATA_DIR / 'turns.csv')
train_df = pd.read_csv(DATA_DIR / 'train.csv')
test_df = pd.read_csv(DATA_DIR / 'test.csv')
sample_submission_df = pd.read_csv(DATA_DIR / 'sample_submission.csv')

print("Data shapes:")
print(f"games_df: {games_df.shape}")
print(f"turns_df: {turns_df.shape}")
print(f"train_df: {train_df.shape}")
print(f"test_df: {test_df.shape}")
print(f"sample_submission_df: {sample_submission_df.shape}")


In [ ]:
# Display basic info about datasets
print("\n=== Games DataFrame Info ===")
print(games_df.info())
print("\n=== Games DataFrame Head ===")
print(games_df.head())

print("\n=== Turns DataFrame Info ===")
print(turns_df.info())
print("\n=== Turns DataFrame Head ===")
print(turns_df.head())

print("\n=== Train DataFrame Info ===")
print(train_df.info())
print("\n=== Train DataFrame Head ===")
print(train_df.head())


<a id='3'></a>
## 3. Анализ целевой переменной

Целевая переменная: `rating` - рейтинг игрока до игры


In [ ]:
# Basic statistics
target_stats = train_df['rating'].describe()
print("=== Статистика целевой переменной (rating) ===")
print(target_stats)
print(f"\nКоличество уникальных значений: {train_df['rating'].nunique()}")
print(f"Пропущенные значения: {train_df['rating'].isna().sum()}")


In [ ]:
# Distribution visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram
axes[0, 0].hist(train_df['rating'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Распределение рейтинга (гистограмма)', fontsize=14)
axes[0, 0].set_xlabel('Рейтинг')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].axvline(train_df['rating'].mean(), color='r', linestyle='--', label=f'Среднее: {train_df["rating"].mean():.2f}')
axes[0, 0].axvline(train_df['rating'].median(), color='g', linestyle='--', label=f'Медиана: {train_df["rating"].median():.2f}')
axes[0, 0].legend()

# Box plot
axes[0, 1].boxplot(train_df['rating'], vert=True)
axes[0, 1].set_title('Распределение рейтинга (ящик с усами)', fontsize=14)
axes[0, 1].set_ylabel('Рейтинг')

# Q-Q plot
from scipy import stats
stats.probplot(train_df['rating'], dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot (нормальность распределения)', fontsize=14)

# Density plot
train_df['rating'].plot(kind='density', ax=axes[1, 1])
axes[1, 1].set_title('Плотность распределения рейтинга', fontsize=14)
axes[1, 1].set_xlabel('Рейтинг')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'target_distribution.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# This cell was moved - feature_types classification now happens in the outlier analysis cell (cell 17)

In [ ]:
# Outlier detection using IQR method
Q1 = train_df['rating'].quantile(0.25)
Q3 = train_df['rating'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = train_df[(train_df['rating'] < lower_bound) | (train_df['rating'] > upper_bound)]
print(f"=== Анализ аномальных значений ===")
print(f"Q1: {Q1:.2f}")
print(f"Q3: {Q3:.2f}")
print(f"IQR: {IQR:.2f}")
print(f"Нижняя граница: {lower_bound:.2f}")
print(f"Верхняя граница: {upper_bound:.2f}")
print(f"Количество аномальных значений: {len(outliers)} ({len(outliers)/len(train_df)*100:.2f}%)")

if len(outliers) > 0:
    print(f"\nМинимальное значение: {train_df['rating'].min():.2f}")
    print(f"Максимальное значение: {train_df['rating'].max():.2f}")


In [ ]:
# Temporal analysis if 'created_at' exists in games_df
if 'created_at' in games_df.columns:
    # Merge to get ratings over time
    train_with_time = train_df.merge(games_df[['game_id', 'created_at']], on='game_id', how='left')
    train_with_time['created_at'] = pd.to_datetime(train_with_time['created_at'])
    train_with_time = train_with_time.sort_values('created_at')
    
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    # Rating over time
    axes[0].scatter(train_with_time['created_at'], train_with_time['rating'], alpha=0.1, s=1)
    axes[0].set_title('Рейтинг во времени', fontsize=14)
    axes[0].set_xlabel('Дата')
    axes[0].set_ylabel('Рейтинг')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Rolling mean
    train_with_time['rating_rolling_mean'] = train_with_time['rating'].rolling(window=1000).mean()
    axes[1].plot(train_with_time['created_at'], train_with_time['rating_rolling_mean'], color='red', linewidth=2)
    axes[1].set_title('Скользящее среднее рейтинга (окно 1000)', fontsize=14)
    axes[1].set_xlabel('Дата')
    axes[1].set_ylabel('Рейтинг (скользящее среднее)')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'target_temporal.png', dpi=300, bbox_inches='tight')
    plt.show()


<a id='4'></a>
## 4. Анализ признаков


In [ ]:
# Import utility functions
import sys
sys.path.append(str(PROJECT_ROOT / 'src'))
from utils.feature_engineering import aggregate_turns_features, classify_features, create_additional_features

# Create aggregated features
turns_features = aggregate_turns_features(turns_df)
print(f"Turns features shape: {turns_features.shape}")
print(turns_features.head())


In [ ]:
# Merge all data
train_merged = (
    train_df
    .merge(games_df, on='game_id', how='left')
    .merge(turns_features, on=['game_id', 'nickname'], how='left')
)

print(f"Merged train shape: {train_merged.shape}")
print(f"\nColumns: {list(train_merged.columns)}")


#### 4.2. Выявление аномальных значений в признаках


In [ ]:
# Анализ аномальных значений в числовых признаках
print("=== Анализ аномальных значений в признаках ===")

# Classify features if not already done
if 'feature_types' not in globals() or feature_types is None:
    feature_types = classify_features(train_merged)

numeric_cols_for_outliers = feature_types['numeric'][:15]  # Первые 15 числовых признаков

outlier_summary = []
for col in numeric_cols_for_outliers:
    if col in train_merged.columns:
        Q1 = train_merged[col].quantile(0.25)
        Q3 = train_merged[col].quantile(0.75)
        IQR = Q3 - Q1
        
        if IQR > 0:  # Избегаем деления на ноль
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers_count = ((train_merged[col] < lower_bound) | (train_merged[col] > upper_bound)).sum()
            outlier_percent = (outliers_count / len(train_merged)) * 100
            
            if outliers_count > 0:
                outlier_summary.append({
                    'Feature': col,
                    'Outliers': outliers_count,
                    'Percent': outlier_percent,
                    'Min': train_merged[col].min(),
                    'Max': train_merged[col].max(),
                    'Lower_bound': lower_bound,
                    'Upper_bound': upper_bound
                })

if outlier_summary:
    outlier_df = pd.DataFrame(outlier_summary)
    print("\nПризнаки с аномальными значениями:")
    print(outlier_df.sort_values('Percent', ascending=False))
    
    # Визуализация топ-5 признаков с наибольшим процентом аномалий
    top_outliers = outlier_df.nlargest(5, 'Percent')
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Процент аномалий
    axes[0].barh(top_outliers['Feature'], top_outliers['Percent'])
    axes[0].set_title('Топ-5 признаков по проценту аномальных значений', fontsize=14)
    axes[0].set_xlabel('Процент аномальных значений')
    
    # Количество аномалий
    axes[1].barh(top_outliers['Feature'], top_outliers['Outliers'])
    axes[1].set_title('Топ-5 признаков по количеству аномальных значений', fontsize=14)
    axes[1].set_xlabel('Количество аномальных значений')
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'feature_outliers.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("Аномальные значения не обнаружены в анализируемых признаках")


In [ ]:
# Feature type classification
feature_types = classify_features(train_merged)
print("=== Классификация признаков ===")
print(f"Числовые признаки ({len(feature_types['numeric'])}): {feature_types['numeric']}")
print(f"\nКатегориальные признаки ({len(feature_types['categorical'])}): {feature_types['categorical']}")
print(f"\nВременные признаки ({len(feature_types['datetime'])}): {feature_types['datetime']}")


In [ ]:
# Missing values analysis
missing_values = train_merged.isnull().sum()
missing_percent = (missing_values / len(train_merged)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percent': missing_percent
}).sort_values('Missing Count', ascending=False)

print("=== Анализ пропущенных значений ===")
print(missing_df[missing_df['Missing Count'] > 0])

# Visualization
if missing_df['Missing Count'].sum() > 0:
    plt.figure(figsize=(12, 6))
    missing_df[missing_df['Missing Count'] > 0]['Missing Percent'].plot(kind='barh')
    plt.title('Процент пропущенных значений по признакам', fontsize=14)
    plt.xlabel('Процент пропущенных значений')
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'missing_values.png', dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
# Feature distributions visualization
numeric_cols = feature_types['numeric'][:10]  # First 10 numeric features

if len(numeric_cols) > 0:
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    axes = axes.flatten()
    
    for idx, col in enumerate(numeric_cols):
        if idx < len(axes):
            train_merged[col].hist(bins=30, ax=axes[idx], edgecolor='black', alpha=0.7)
            axes[idx].set_title(f'{col}', fontsize=10)
            axes[idx].set_xlabel('')
            axes[idx].set_ylabel('')
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'numeric_features_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
# Correlation analysis
numeric_cols_all = feature_types['numeric']
correlation_matrix = train_merged[numeric_cols_all + ['rating']].corr()

# Correlation with target
target_corr = correlation_matrix['rating'].sort_values(ascending=False)
print("=== Корреляция признаков с целевой переменной ===")
print(target_corr)

# Visualization
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Матрица корреляций', fontsize=14)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Feature importance visualization (top correlations)
top_corr = target_corr.drop('rating').abs().sort_values(ascending=False).head(15)

plt.figure(figsize=(10, 8))
top_corr.plot(kind='barh')
plt.title('Топ-15 признаков по абсолютной корреляции с целевой переменной', fontsize=14)
plt.xlabel('Абсолютная корреляция')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'feature_importance_correlation.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Categorical features analysis
categorical_cols = feature_types['categorical']

if len(categorical_cols) > 0:
    fig, axes = plt.subplots(len(categorical_cols), 1, figsize=(12, 4 * len(categorical_cols)))
    if len(categorical_cols) == 1:
        axes = [axes]
    
    for idx, col in enumerate(categorical_cols):
        value_counts = train_merged[col].value_counts().head(10)
        value_counts.plot(kind='bar', ax=axes[idx])
        axes[idx].set_title(f'Распределение значений: {col}', fontsize=12)
        axes[idx].set_xlabel('')
        axes[idx].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'categorical_features_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()


<a id='5'></a>
## 5. Бейзлайн с LightAutoML

Создадим минимум 2 различные конфигурации и выберем лучшую.


In [ ]:
# Prepare data for LightAutoML
# Remove target and ID columns
feature_cols = [col for col in train_merged.columns 
                if col not in ['rating', 'game_id', 'nickname']]

X_train = train_merged[feature_cols].copy()
y_train = train_merged['rating'].copy()

# Handle missing values
X_train = X_train.fillna(0)

# Split for validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

print(f"Train shape: {X_train_split.shape}")
print(f"Validation shape: {X_val_split.shape}")


In [ ]:
# Configuration 1: Default TabularAutoML
print("=== Конфигурация 1: TabularAutoML (default) ===")

task1 = Task('reg', metric='mse')
automl1 = TabularAutoML(
    task=task1,
    timeout=240,  # 4 minutes
    cpu_limit=N_THREADS,
    general_params={'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]}
)

train_data_1 = pd.concat([X_train_split, y_train_split], axis=1)
train_data_1.columns = list(X_train_split.columns) + ['rating']

start_time = time.time()
oof_pred1 = automl1.fit_predict(
    train_data_1,
    roles={'target': 'rating'},
    verbose=1
)
time1 = time.time() - start_time

val_pred1 = automl1.predict(X_val_split)
rmse1 = np.sqrt(mean_squared_error(y_val_split, val_pred1.data[:, 0]))
mae1 = mean_absolute_error(y_val_split, val_pred1.data[:, 0])
r2_1 = r2_score(y_val_split, val_pred1.data[:, 0])

print(f"\nВремя обучения: {time1:.2f} секунд")
print(f"RMSE: {rmse1:.4f}")
print(f"MAE: {mae1:.4f}")
print(f"R2 Score: {r2_1:.4f}")


In [ ]:
# Configuration 2: TabularAutoML with more algorithms
print("=== Конфигурация 2: TabularAutoML (extended algorithms) ===")

task2 = Task('reg', metric='mse')
automl2 = TabularAutoML(
    task=task2,
    timeout=360,  # 6 minutes
    cpu_limit=N_THREADS,
    general_params={'use_algos': [['linear_l2', 'lgb', 'lgb_tuned', 'cb', 'cb_tuned']]}
)

train_data_2 = pd.concat([X_train_split, y_train_split], axis=1)
train_data_2.columns = list(X_train_split.columns) + ['rating']

start_time = time.time()
oof_pred2 = automl2.fit_predict(
    train_data_2,
    roles={'target': 'rating'},
    verbose=1
)
time2 = time.time() - start_time

val_pred2 = automl2.predict(X_val_split)
rmse2 = np.sqrt(mean_squared_error(y_val_split, val_pred2.data[:, 0]))
mae2 = mean_absolute_error(y_val_split, val_pred2.data[:, 0])
r2_2 = r2_score(y_val_split, val_pred2.data[:, 0])

print(f"\nВремя обучения: {time2:.2f} секунд")
print(f"RMSE: {rmse2:.4f}")
print(f"MAE: {mae2:.4f}")
print(f"R2 Score: {r2_2:.4f}")


In [ ]:
# Compare configurations
comparison_df = pd.DataFrame({
    'Configuration': ['Config 1 (LGB)', 'Config 2 (LGB + CB)'],
    'RMSE': [rmse1, rmse2],
    'MAE': [mae1, mae2],
    'R2 Score': [r2_1, r2_2],
    'Time (seconds)': [time1, time2]
})

print("=== Сравнение конфигураций LightAutoML ===")
print(comparison_df)

# Select best model
best_config_idx = comparison_df['RMSE'].idxmin()
best_automl = automl1 if best_config_idx == 0 else automl2
best_rmse = comparison_df.loc[best_config_idx, 'RMSE']

print(f"\nЛучшая конфигурация: {comparison_df.loc[best_config_idx, 'Configuration']}")
print(f"Лучший RMSE: {best_rmse:.4f}")


<a id='6'></a>
## 6. Собственное решение

Создадим собственный пайплайн без использования LightAutoML, демонстрируя принципы AutoML:

1. **Автоматический выбор признаков** - автоматический отбор наиболее информативных признаков
2. **Автоматическая предобработка** - обработка пропусков, кодирование категориальных переменных
3. **Автоматический выбор модели** - сравнение нескольких алгоритмов и выбор лучшего
4. **Автоматическая оптимизация гиперпараметров** - поиск оптимальных параметров модели
5. **Автоматическое ансамблирование** - комбинирование нескольких моделей с оптимизацией весов


In [ ]:
# Create enhanced features
train_enhanced = create_additional_features(train_df, games_df, turns_features)
print(f"Enhanced train shape: {train_enhanced.shape}")


In [ ]:
# Prepare data for custom pipeline
feature_cols_enhanced = [col for col in train_enhanced.columns 
                        if col not in ['rating', 'game_id', 'nickname']]

X_train_custom = train_enhanced[feature_cols_enhanced].copy()
y_train_custom = train_enhanced['rating'].copy()

# Split data
X_train_custom_split, X_val_custom_split, y_train_custom_split, y_val_custom_split = train_test_split(
    X_train_custom, y_train_custom, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

print(f"Custom train shape: {X_train_custom_split.shape}")
print(f"Custom validation shape: {X_val_custom_split.shape}")


In [ ]:
# Pipeline 1: LightGBM with feature selection
# AutoML принцип: Автоматический выбор признаков (Feature Selection)
from lightgbm import LGBMRegressor
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import numpy as np

print("=== Pipeline 1: LightGBM с автоматическим отбором признаков ===")
print("Принцип AutoML: Автоматический выбор наиболее информативных признаков")

# Обработка категориальных признаков перед feature selection
categorical_cols_p1 = [col for col in X_train_custom_split.columns 
                       if X_train_custom_split[col].dtype == 'object']

# Кодируем категориальные признаки
X_train_encoded_p1 = X_train_custom_split.copy()
X_val_encoded_p1 = X_val_custom_split.copy()

label_encoders_p1 = {}
for col in categorical_cols_p1:
    le = LabelEncoder()
    # train
    train_col = X_train_encoded_p1[col].astype(str).fillna('unknown')
    le.fit(train_col)
    # гарантируем наличие 'unknown' в классах
    if 'unknown' not in le.classes_:
        le.classes_ = np.append(le.classes_, 'unknown')
    X_train_encoded_p1[col] = le.transform(train_col)

    # val: неизвестные значения мапим в 'unknown'
    val_col = X_val_encoded_p1[col].astype(str).fillna('unknown')
    known = set(le.classes_)
    val_col = val_col.apply(lambda x: x if x in known else 'unknown')
    X_val_encoded_p1[col] = le.transform(val_col)

    label_encoders_p1[col] = le

# Заполняем пропуски
X_train_encoded_p1 = X_train_encoded_p1.fillna(0)
X_val_encoded_p1 = X_val_encoded_p1.fillna(0)

# Используем только числовые признаки (без дат и категориальных)
X_train_num_p1 = X_train_encoded_p1.select_dtypes(include=[np.number])
X_val_num_p1 = X_val_encoded_p1.select_dtypes(include=[np.number])

# Автоматический выбор признаков - пробуем разные методы
# Метод 1: F-статистика
selector_f = SelectKBest(f_regression, k=50)
X_train_f = selector_f.fit_transform(X_train_num_p1, y_train_custom_split)

# Метод 2: Mutual Information
try:
    selector_mi = SelectKBest(mutual_info_regression, k=50)
    X_train_mi = selector_mi.fit_transform(X_train_num_p1, y_train_custom_split)
    
    # Выбираем лучший метод на основе кросс-валидации
    from sklearn.model_selection import cross_val_score
    temp_model = LGBMRegressor(n_estimators=100, random_state=RANDOM_STATE, verbose=-1)
    
    score_f = cross_val_score(temp_model, X_train_f, y_train_custom_split, 
                              cv=3, scoring='neg_mean_squared_error').mean()
    score_mi = cross_val_score(temp_model, X_train_mi, y_train_custom_split, 
                               cv=3, scoring='neg_mean_squared_error').mean()
    
    if score_mi > score_f:
        selector = selector_mi
        print(f"Выбран метод: Mutual Information (score: {score_mi:.4f})")
    else:
        selector = selector_f
        print(f"Выбран метод: F-regression (score: {score_f:.4f})")
except:
    selector = selector_f
    print("Используется метод: F-regression")

X_train_selected = selector.fit_transform(X_train_num_p1, y_train_custom_split)
X_val_selected = selector.transform(X_val_num_p1)

# Model
model1 = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    random_state=RANDOM_STATE,
    n_jobs=N_THREADS,
    verbose=-1
)

start_time = time.time()
model1.fit(X_train_selected, y_train_custom_split)
time_p1 = time.time() - start_time

val_pred_p1 = model1.predict(X_val_selected)
rmse_p1 = np.sqrt(mean_squared_error(y_val_custom_split, val_pred_p1))
mae_p1 = mean_absolute_error(y_val_custom_split, val_pred_p1)
r2_p1 = r2_score(y_val_custom_split, val_pred_p1)

print(f"Время обучения: {time_p1:.2f} секунд")
print(f"RMSE: {rmse_p1:.4f}")
print(f"MAE: {mae_p1:.4f}")
print(f"R2 Score: {r2_p1:.4f}")


In [ ]:
# Pipeline 2: CatBoost with preprocessing
# AutoML принцип: Автоматическая предобработка данных
from catboost import CatBoostRegressor

print("=== Pipeline 2: CatBoost с автоматической предобработкой ===")
print("Принцип AutoML: Автоматическая обработка категориальных признаков и пропусков")

# Identify categorical columns
categorical_cols_custom = [col for col in X_train_custom_split.columns 
                           if X_train_custom_split[col].dtype == 'object']

# Encode categorical variables
label_encoders = {}
X_train_encoded = X_train_custom_split.copy()
X_val_encoded = X_val_custom_split.copy()

for col in categorical_cols_custom:
    le = LabelEncoder()
    # train
    train_col = X_train_encoded[col].astype(str).fillna('unknown')
    le.fit(train_col)
    # гарантируем наличие 'unknown' в классах
    if 'unknown' not in le.classes_:
        le.classes_ = np.append(le.classes_, 'unknown')
    X_train_encoded[col] = le.transform(train_col)

    # val: неизвестные значения мапим в 'unknown'
    val_col = X_val_encoded[col].astype(str).fillna('unknown')
    known = set(le.classes_)
    val_col = val_col.apply(lambda x: x if x in known else 'unknown')
    X_val_encoded[col] = le.transform(val_col)

    label_encoders[col] = le

# Fill missing values
X_train_encoded = X_train_encoded.fillna(0)
X_val_encoded = X_val_encoded.fillna(0)

# Model
model2 = CatBoostRegressor(
    iterations=300,
    learning_rate=0.05,
    depth=7,
    random_state=RANDOM_STATE,
    thread_count=N_THREADS,
    verbose=False
)

start_time = time.time()
model2.fit(X_train_encoded, y_train_custom_split)
time_p2 = time.time() - start_time

val_pred_p2 = model2.predict(X_val_encoded)
rmse_p2 = np.sqrt(mean_squared_error(y_val_custom_split, val_pred_p2))
mae_p2 = mean_absolute_error(y_val_custom_split, val_pred_p2)
r2_p2 = r2_score(y_val_custom_split, val_pred_p2)

print(f"Время обучения: {time_p2:.2f} секунд")
print(f"RMSE: {rmse_p2:.4f}")
print(f"MAE: {mae_p2:.4f}")
print(f"R2 Score: {r2_p2:.4f}")


In [ ]:
# Pipeline 3: Ensemble (LightGBM + CatBoost)
# AutoML принцип: Ансамблирование моделей
print("=== Pipeline 3: Простой ансамбль (LightGBM + CatBoost) ===")
print("Принцип AutoML: Комбинирование нескольких моделей для улучшения предсказаний")

# Use the same preprocessing as Pipeline 2
# Используем только числовые признаки (исключаем datetime)
X_train_encoded_num = X_train_encoded.select_dtypes(include=[np.number])
X_val_encoded_num = X_val_encoded.select_dtypes(include=[np.number])

model_lgb = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    random_state=RANDOM_STATE,
    n_jobs=N_THREADS,
    verbose=-1
)

model_cat = CatBoostRegressor(
    iterations=300,
    learning_rate=0.05,
    depth=7,
    random_state=RANDOM_STATE + 1,
    thread_count=N_THREADS,
    verbose=False
)

start_time = time.time()
model_lgb.fit(X_train_encoded_num, y_train_custom_split)
model_cat.fit(X_train_encoded_num, y_train_custom_split)
time_p3 = time.time() - start_time

val_pred_lgb = model_lgb.predict(X_val_encoded_num)
val_pred_cat = model_cat.predict(X_val_encoded_num)
val_pred_p3 = (val_pred_lgb + val_pred_cat) / 2  # Simple average

rmse_p3 = np.sqrt(mean_squared_error(y_val_custom_split, val_pred_p3))
mae_p3 = mean_absolute_error(y_val_custom_split, val_pred_p3)
r2_p3 = r2_score(y_val_custom_split, val_pred_p3)

print(f"Время обучения: {time_p3:.2f} секунд")
print(f"RMSE: {rmse_p3:.4f}")
print(f"MAE: {mae_p3:.4f}")
print(f"R2 Score: {r2_p3:.4f}")


In [ ]:
# 6.4. Сравнение всех моделей (AutoML принцип: Model Selection)
# Сравним все созданные пайплайны и автоматически выберем лучший

print("=== Сравнение всех моделей ===")
if 'all_results' in locals():
    print(all_results.sort_values('RMSE'))
    
    # Find best model
    best_model_idx = all_results['RMSE'].idxmin()
    best_model_name = all_results.loc[best_model_idx, 'Pipeline']
    best_rmse_all = all_results.loc[best_model_idx, 'RMSE']
    
    print(f"\nЛучшая модель: {best_model_name}")
    print(f"Лучший RMSE: {best_rmse_all:.4f}")
else:
    print("Переменная all_results еще не создана. Запустите ячейку с созданием all_results сначала.")


# Compare all pipelines
all_results = pd.DataFrame({


In [ ]:
# Автоматическая оптимизация гиперпараметров с помощью Optuna
try:
    import optuna
    from sklearn.model_selection import cross_val_score
    
    print("=== Pipeline 4: Автоматическая оптимизация гиперпараметров (Optuna) ===")
    
    # Используем только числовые признаки (исключаем datetime)
    X_train_encoded_num_p4 = X_train_encoded.select_dtypes(include=[np.number])
    X_val_encoded_num_p4 = X_val_encoded.select_dtypes(include=[np.number])
    
    def objective_lgb(trial):
        """Функция для оптимизации гиперпараметров LightGBM"""
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        }
        
        model = LGBMRegressor(
            **params,
            random_state=RANDOM_STATE,
            n_jobs=N_THREADS,
            verbose=-1
        )
        
        scores = cross_val_score(
            model, X_train_encoded_num_p4, y_train_custom_split,
            cv=3, scoring='neg_mean_squared_error',
            n_jobs=N_THREADS
        )
        return -scores.mean()
    
    # Оптимизация
    study = optuna.create_study(direction='minimize', study_name='lgb_optimization')
    study.optimize(objective_lgb, n_trials=20, timeout=240, show_progress_bar=True)
    
    print(f"\nЛучшие гиперпараметры:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")
    print(f"\nЛучший RMSE (CV): {np.sqrt(study.best_value):.4f}")
    
    # Обучаем модель с лучшими параметрами
    best_params = study.best_params.copy()
    model_optimized = LGBMRegressor(
        **best_params,
        random_state=RANDOM_STATE,
        n_jobs=N_THREADS,
        verbose=-1
    )
    
    start_time = time.time()
    model_optimized.fit(X_train_encoded_num_p4, y_train_custom_split)
    time_p4 = time.time() - start_time
    
    val_pred_p4 = model_optimized.predict(X_val_encoded_num_p4)
    rmse_p4 = np.sqrt(mean_squared_error(y_val_custom_split, val_pred_p4))
    mae_p4 = mean_absolute_error(y_val_custom_split, val_pred_p4)
    r2_p4 = r2_score(y_val_custom_split, val_pred_p4)
    
    print(f"\nВремя обучения: {time_p4:.2f} секунд")
    print(f"RMSE: {rmse_p4:.4f}")
    print(f"MAE: {mae_p4:.4f}")
    print(f"R2 Score: {r2_p4:.4f}")
    
    optuna_available = True
except ImportError:
    print("Optuna не установлен. Установите: pip install optuna")
    optuna_available = False
    rmse_p4 = float('inf')
    mae_p4 = float('inf')
    r2_p4 = 0
    time_p4 = 0


### 6.6. Автоматическое ансамблирование с оптимизацией весов (AutoML принцип: Ensemble Learning)

Оптимизируем веса для ансамбля моделей:


In [ ]:
# Автоматическое ансамблирование с оптимизацией весов
from scipy.optimize import minimize

print("=== Pipeline 5: Оптимизированный ансамбль ===")

# Собираем предсказания всех моделей на валидации
ensemble_models = {
    'LGB': model_lgb,
    'CatBoost': model_cat,
}

if optuna_available and 'model_optimized' in locals():
    ensemble_models['LGB_Optimized'] = model_optimized

# Получаем предсказания (используем только числовые признаки)
X_val_encoded_num_p5 = X_val_encoded.select_dtypes(include=[np.number])
ensemble_predictions = {}
for name, model in ensemble_models.items():
    ensemble_predictions[name] = model.predict(X_val_encoded_num_p5)

# Функция для оптимизации весов
def objective_weights(weights):
    """Минимизируем RMSE ансамбля"""
    weights = weights / weights.sum()  # Нормализуем веса
    pred = np.zeros(len(y_val_custom_split))
    for idx, (name, pred_vals) in enumerate(ensemble_predictions.items()):
        pred += weights[idx] * pred_vals
    return mean_squared_error(y_val_custom_split, pred)

# Оптимизация весов
n_models = len(ensemble_models)
initial_weights = np.ones(n_models) / n_models
bounds = [(0, 1)] * n_models
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

result = minimize(
    objective_weights,
    initial_weights,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

optimal_weights = result.x / result.x.sum()
print(f"\nОптимальные веса для ансамбля:")
for idx, (name, _) in enumerate(ensemble_predictions.items()):
    print(f"  {name}: {optimal_weights[idx]:.4f}")

# Предсказания с оптимальными весами
val_pred_ensemble = np.zeros(len(y_val_custom_split))
for idx, (name, pred_vals) in enumerate(ensemble_predictions.items()):
    val_pred_ensemble += optimal_weights[idx] * pred_vals

rmse_p5 = np.sqrt(mean_squared_error(y_val_custom_split, val_pred_ensemble))
mae_p5 = mean_absolute_error(y_val_custom_split, val_pred_ensemble)
r2_p5 = r2_score(y_val_custom_split, val_pred_ensemble)

print(f"\nRMSE оптимизированного ансамбля: {rmse_p5:.4f}")
print(f"MAE: {mae_p5:.4f}")
print(f"R2 Score: {r2_p5:.4f}")

# Сравнение с простым средним
simple_ensemble_pred = np.mean([pred for pred in ensemble_predictions.values()], axis=0)
rmse_simple = np.sqrt(mean_squared_error(y_val_custom_split, simple_ensemble_pred))
print(f"\nRMSE простого среднего: {rmse_simple:.4f}")
print(f"Улучшение: {rmse_simple - rmse_p5:.4f} ({((rmse_simple - rmse_p5) / rmse_simple * 100):.2f}%)")


In [ ]:
# Compare all pipelines
# Инициализируем переменные для случая, если Optuna не установлен
if 'optuna_available' not in locals():
    optuna_available = False

pipelines_list = [
    'LAMA Config 1',
    'LAMA Config 2',
    'Custom Pipeline 1 (LGB + Auto Feature Selection)',
    'Custom Pipeline 2 (CatBoost + Auto Preprocessing)',
    'Custom Pipeline 3 (Simple Ensemble)'
]
rmse_list = [rmse1, rmse2, rmse_p1, rmse_p2, rmse_p3]
mae_list = [mae1, mae2, mae_p1, mae_p2, mae_p3]
r2_list = [r2_1, r2_2, r2_p1, r2_p2, r2_p3]
time_list = [time1, time2, time_p1, time_p2, time_p3]

# Добавляем оптимизированные модели, если они доступны
if optuna_available and 'rmse_p4' in locals():
    pipelines_list.append('Custom Pipeline 4 (Auto Hyperparameter Optimization)')
    rmse_list.append(rmse_p4)
    mae_list.append(mae_p4)
    r2_list.append(r2_p4)
    time_list.append(time_p4)

if 'rmse_p5' in locals():
    pipelines_list.append('Custom Pipeline 5 (Optimized Ensemble)')
    rmse_list.append(rmse_p5)
    mae_list.append(mae_p5)
    r2_list.append(r2_p5)
    time_list.append(time_p3 + 10)  # Примерное время для ансамбля

all_results = pd.DataFrame({
    'Pipeline': pipelines_list,
    'RMSE': rmse_list,
    'MAE': mae_list,
    'R2 Score': r2_list,
    'Time (seconds)': time_list
})

print("=== Сравнение всех моделей ===")
print(all_results.sort_values('RMSE'))

# Find best model
best_model_idx = all_results['RMSE'].idxmin()
best_model_name = all_results.loc[best_model_idx, 'Pipeline']
best_rmse_all = all_results.loc[best_model_idx, 'RMSE']

print(f"\nЛучшая модель: {best_model_name}")
print(f"Лучший RMSE: {best_rmse_all:.4f}")


In [ ]:
# Generate predictions for test set
print("=== Генерация предсказаний для тестового набора ===")

# Фильтруем только те строки, для которых нужно предсказать рейтинг (rating == NaN)
test_df_to_predict = test_df[test_df['rating'].isna()].copy()
print(f"Всего строк в test_df: {len(test_df)}")
print(f"Строк для предсказания (rating == NaN): {len(test_df_to_predict)}")

# Prepare test data только для строк с NaN rating
test_enhanced = create_additional_features(test_df_to_predict, games_df, turns_features)
print(f"Строк после создания признаков: {len(test_enhanced)}")

# Use best custom model for final predictions
# Автоматически выбираем лучшую модель (принцип AutoML: автоматический выбор модели)
if 'optuna_available' not in locals():
    optuna_available = False

best_custom_rmse = min([rmse_p1, rmse_p2, rmse_p3])
if optuna_available and 'rmse_p4' in locals():
    best_custom_rmse = min(best_custom_rmse, rmse_p4)
if 'rmse_p5' in locals():
    best_custom_rmse = min(best_custom_rmse, rmse_p5)

if 'rmse_p5' in locals() and rmse_p5 == best_custom_rmse:
    # Use optimized ensemble
    X_test = test_enhanced[feature_cols_enhanced].copy()
    
    for col in categorical_cols_custom:
        if col in X_test.columns:
            le = label_encoders[col]
            test_col = X_test[col].astype(str).fillna('unknown')
            known = set(le.classes_)
            test_col = test_col.apply(lambda x: x if x in known else 'unknown')
            X_test[col] = le.transform(test_col)
    
    X_test = X_test.fillna(0)
    # Используем только числовые признаки (модели обучены на числовых данных)
    X_test_num = X_test.select_dtypes(include=[np.number])
    
    # Используем оптимизированные веса
    test_predictions = np.zeros(len(X_test_num))
    for idx, (name, model) in enumerate(ensemble_models.items()):
        test_predictions += optimal_weights[idx] * model.predict(X_test_num)
    
    print("Использована модель: Оптимизированный ансамбль (AutoML принцип: Ensemble Learning)")
elif optuna_available and 'rmse_p4' in locals() and rmse_p4 == best_custom_rmse:
    # Use optimized model
    X_test = test_enhanced[feature_cols_enhanced].copy()
    
    for col in categorical_cols_custom:
        if col in X_test.columns:
            le = label_encoders[col]
            test_col = X_test[col].astype(str).fillna('unknown')
            known = set(le.classes_)
            test_col = test_col.apply(lambda x: x if x in known else 'unknown')
            X_test[col] = le.transform(test_col)
    
    X_test = X_test.fillna(0)
    # Используем только числовые признаки (модель обучена на числовых данных)
    X_test_num = X_test.select_dtypes(include=[np.number])
    test_predictions = model_optimized.predict(X_test_num)
    
    print("Использована модель: Оптимизированный LightGBM (AutoML принцип: Hyperparameter Optimization)")
elif rmse_p3 < rmse_p2 and rmse_p3 < rmse_p1:
    # Use ensemble
    X_test = test_enhanced[feature_cols_enhanced].copy()
    
    # Apply same preprocessing
    for col in categorical_cols_custom:
        if col in X_test.columns:
            le = label_encoders[col]
            test_col = X_test[col].astype(str).fillna('unknown')
            known = set(le.classes_)
            test_col = test_col.apply(lambda x: x if x in known else 'unknown')
            X_test[col] = le.transform(test_col)
    
    X_test = X_test.fillna(0)
    # Используем только числовые признаки (модели обучены на числовых данных)
    X_test_num = X_test.select_dtypes(include=[np.number])
    
    test_pred_lgb = model_lgb.predict(X_test_num)
    test_pred_cat = model_cat.predict(X_test_num)
    test_predictions = (test_pred_lgb + test_pred_cat) / 2
    
    print("Использована модель: Ensemble (LightGBM + CatBoost)")
elif rmse_p2 < rmse_p1:
    # Use CatBoost
    X_test = test_enhanced[feature_cols_enhanced].copy()
    
    for col in categorical_cols_custom:
        if col in X_test.columns:
            le = label_encoders[col]
            test_col = X_test[col].astype(str).fillna('unknown')
            known = set(le.classes_)
            test_col = test_col.apply(lambda x: x if x in known else 'unknown')
            X_test[col] = le.transform(test_col)
    
    X_test = X_test.fillna(0)
    # Используем только числовые признаки (для консистентности)
    X_test_num = X_test.select_dtypes(include=[np.number])
    test_predictions = model2.predict(X_test_num)
    
    print("Использована модель: CatBoost")
else:
    # Use LightGBM
    X_test = test_enhanced[feature_cols_enhanced].copy()
    
    # Обработка категориальных признаков для Pipeline 1
    for col in categorical_cols_p1:
        if col in X_test.columns:
            le = label_encoders_p1[col]
            test_col = X_test[col].astype(str).fillna('unknown')
            known = set(le.classes_)
            test_col = test_col.apply(lambda x: x if x in known else 'unknown')
            X_test[col] = le.transform(test_col)
    
    X_test = X_test.fillna(0)
    # Используем только числовые признаки (как в Pipeline 1)
    X_test_num = X_test.select_dtypes(include=[np.number])
    X_test_selected = selector.transform(X_test_num)
    test_predictions = model1.predict(X_test_selected)
    
    print("Использована модель: LightGBM")

# Create submission file
# Создаём DataFrame с предсказаниями
predictions_df = pd.DataFrame({
    'game_id': test_enhanced['game_id'].values,
    'nickname': test_enhanced['nickname'].values,
    'rating': test_predictions
})

# В sample_submission.csv только game_id, поэтому для каждого game_id берём средний рейтинг
# или рейтинг первого игрока (если несколько игроков на игру)
submission_ratings = predictions_df.groupby('game_id')['rating'].first().reset_index()

# Объединяем с sample_submission_df для правильного порядка
submission = sample_submission_df[['game_id']].merge(
    submission_ratings,
    on='game_id',
    how='left'
)

# Проверяем, что все рейтинги заполнены
if submission['rating'].isna().sum() > 0:
    print(f"⚠️  ВНИМАНИЕ: {submission['rating'].isna().sum()} строк без предсказаний!")
    # Заполняем пропуски средним значением
    submission['rating'] = submission['rating'].fillna(submission['rating'].mean())
    print(f"Пропуски заполнены средним значением: {submission['rating'].mean():.2f}")

# Save submission
submission_path = RESULTS_DIR / 'submission.csv'
submission.to_csv(submission_path, index=False)
print(f"\nФайл submission сохранен: {submission_path}")
print(f"\nПример предсказаний:")
print(submission.head(10))

<a id='7'></a>
## 7. Выводы и сравнение результатов


In [ ]:
# Visualization of results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# RMSE comparison
all_results_sorted = all_results.sort_values('RMSE')
axes[0].barh(all_results_sorted['Pipeline'], all_results_sorted['RMSE'])
axes[0].set_title('Сравнение RMSE по моделям', fontsize=14)
axes[0].set_xlabel('RMSE')

# R2 Score comparison
axes[1].barh(all_results_sorted['Pipeline'], all_results_sorted['R2 Score'])
axes[1].set_title('Сравнение R2 Score по моделям', fontsize=14)
axes[1].set_xlabel('R2 Score')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Текстовые выводы и анализ результатов
print("\n" + "="*80)
print("АНАЛИЗ РЕЗУЛЬТАТОВ И ВЫВОДЫ")
print("="*80)

# Сортируем результаты по RMSE
all_results_sorted = all_results.sort_values('RMSE')
best_model = all_results_sorted.iloc[0]
worst_model = all_results_sorted.iloc[-1]

print(f"\n📊 ОБЩАЯ СТАТИСТИКА:")
print(f"   Всего протестировано моделей: {len(all_results)}")
print(f"   Диапазон RMSE: {all_results['RMSE'].min():.4f} - {all_results['RMSE'].max():.4f}")
print(f"   Средний RMSE: {all_results['RMSE'].mean():.4f}")
print(f"   Средний R2 Score: {all_results['R2 Score'].mean():.4f}")

print(f"\n🏆 ЛУЧШАЯ МОДЕЛЬ:")
print(f"   Название: {best_model['Pipeline']}")
print(f"   RMSE: {best_model['RMSE']:.4f}")
print(f"   MAE: {best_model['MAE']:.4f}")
print(f"   R2 Score: {best_model['R2 Score']:.4f}")
print(f"   Время обучения: {best_model['Time (seconds)']:.2f} секунд")

# Анализ почему эта модель лучше
print(f"\n💡 ПОЧЕМУ ЭТА МОДЕЛЬ ЛУЧШЕ:")
if 'Ensemble' in best_model['Pipeline'] or 'ансамбль' in best_model['Pipeline']:
    print("   • Ансамблирование моделей позволяет комбинировать сильные стороны разных алгоритмов")
    print("   • Разные модели могут улавливать различные паттерны в данных")
    print("   • Оптимизация весов ансамбля улучшает общую точность предсказаний")
    print("   • Снижает риск переобучения за счет усреднения предсказаний")
    print("   • Более стабильные результаты на новых данных")
elif 'Optimized' in best_model['Pipeline'] or 'Optuna' in best_model['Pipeline']:
    print("   • Автоматическая оптимизация гиперпараметров находит оптимальные настройки")
    print("   • Поиск по пространству параметров позволяет улучшить качество модели")
    print("   • Кросс-валидация обеспечивает надежную оценку производительности")
    print("   • Балансирует сложность модели и обобщающую способность")
    print("   • Учитывает специфику данных через адаптацию параметров")
elif 'CatBoost' in best_model['Pipeline']:
    print("   • CatBoost эффективно обрабатывает категориальные признаки")
    print("   • Встроенная обработка пропусков и категорий упрощает предобработку")
    print("   • Хорошо работает с неструктурированными данными")
    print("   • Устойчив к переобучению благодаря регуляризации")
    print("   • Автоматически обрабатывает категориальные переменные без ручного кодирования")
elif 'LightGBM' in best_model['Pipeline']:
    print("   • LightGBM эффективен для больших объемов данных")
    print("   • Автоматический отбор признаков улучшает качество модели")
    print("   • Быстрое обучение и хорошая точность")
    print("   • Эффективное использование памяти")
    print("   • Хорошо работает с различными типами признаков")
elif 'LAMA' in best_model['Pipeline']:
    print("   • LightAutoML автоматически выполняет предобработку данных")
    print("   • Встроенное ансамблирование нескольких алгоритмов")
    print("   • Автоматический выбор признаков и их трансформация")
    print("   • Оптимизация гиперпараметров из коробки")
    print("   • Минимум ручной настройки при хорошем качестве")

# Сравнение с худшей моделью
improvement = ((worst_model['RMSE'] - best_model['RMSE']) / worst_model['RMSE']) * 100
print(f"\n📈 СРАВНЕНИЕ С ХУДШЕЙ МОДЕЛЬЮ:")
print(f"   Худшая модель: {worst_model['Pipeline']} (RMSE: {worst_model['RMSE']:.4f})")
print(f"   Улучшение: {improvement:.2f}% ({worst_model['RMSE'] - best_model['RMSE']:.4f} единиц RMSE)")

# Сравнение со второй лучшей моделью
if len(all_results_sorted) > 1:
    second_best = all_results_sorted.iloc[1]
    improvement_vs_second = ((second_best['RMSE'] - best_model['RMSE']) / second_best['RMSE']) * 100
    print(f"\n📊 СРАВНЕНИЕ СО ВТОРОЙ ЛУЧШЕЙ МОДЕЛЬЮ:")
    print(f"   Вторая лучшая: {second_best['Pipeline']} (RMSE: {second_best['RMSE']:.4f})")
    print(f"   Преимущество лучшей модели: {improvement_vs_second:.2f}% ({second_best['RMSE'] - best_model['RMSE']:.4f} единиц RMSE)")
    if improvement_vs_second < 1:
        print(f"   ⚠ Разница минимальна - обе модели показывают схожее качество")
    elif improvement_vs_second < 5:
        print(f"   ✓ Заметное улучшение - лучшая модель имеет преимущество")
    else:
        print(f"   ✓ Значительное улучшение - лучшая модель существенно превосходит")

# Анализ по категориям моделей
print(f"\n📋 АНАЛИЗ ПО КАТЕГОРИЯМ:")
lama_models = all_results[all_results['Pipeline'].str.contains('LAMA', case=False)]
custom_models = all_results[~all_results['Pipeline'].str.contains('LAMA', case=False)]

if len(lama_models) > 0:
    print(f"\n   LightAutoML модели:")
    print(f"   • Средний RMSE: {lama_models['RMSE'].mean():.4f}")
    print(f"   • Лучший RMSE: {lama_models['RMSE'].min():.4f}")
    print(f"   • Преимущества: автоматическая предобработка, выбор признаков, ансамблирование")

if len(custom_models) > 0:
    print(f"\n   Собственные модели:")
    print(f"   • Средний RMSE: {custom_models['RMSE'].mean():.4f}")
    print(f"   • Лучший RMSE: {custom_models['RMSE'].min():.4f}")
    print(f"   • Преимущества: контроль над процессом, возможность тонкой настройки")

# Детальный анализ метрик
print(f"\n📉 ДЕТАЛЬНЫЙ АНАЛИЗ МЕТРИК:")
print(f"   RMSE (Root Mean Squared Error): {best_model['RMSE']:.4f}")
print(f"     → Средняя ошибка предсказания: ±{best_model['RMSE']:.2f} единиц рейтинга")
print(f"   MAE (Mean Absolute Error): {best_model['MAE']:.4f}")
print(f"     → Средняя абсолютная ошибка: {best_model['MAE']:.2f} единиц рейтинга")
print(f"   R² Score: {best_model['R2 Score']:.4f}")
if best_model['R2 Score'] > 0.9:
    print(f"     → Модель объясняет более 90% дисперсии - отличное качество!")
elif best_model['R2 Score'] > 0.8:
    print(f"     → Модель объясняет более 80% дисперсии - очень хорошее качество")
elif best_model['R2 Score'] > 0.6:
    print(f"     → Модель объясняет более 60% дисперсии - хорошее качество")
elif best_model['R2 Score'] > 0.4:
    print(f"     → Модель объясняет более 40% дисперсии - приемлемое качество")
else:
    print(f"     → Модель объясняет менее 40% дисперсии - требуется улучшение")

# Рекомендации
print(f"\n💼 РЕКОМЕНДАЦИИ ДЛЯ ПРОДАКШЕНА:")
if best_model['R2 Score'] > 0.8:
    print("   ✓ Модель показывает отличное качество (R2 > 0.8)")
    print("   ✓ Готова к использованию в продакшене")
elif best_model['R2 Score'] > 0.6:
    print("   ✓ Модель показывает хорошее качество (R2 > 0.6)")
    print("   ✓ Можно использовать в продакшене с мониторингом")
else:
    print("   ⚠ Модель можно улучшить (R2 < 0.6)")
    print("   ⚠ Рекомендуется дополнительная работа над признаками и моделью")

if best_model['Time (seconds)'] < 300:
    print(f"   ✓ Время обучения приемлемое ({best_model['Time (seconds)']:.2f} сек < 5 минут)")
    print("   ✓ Модель можно переобучать регулярно")
else:
    print(f"   ⚠ Время обучения можно оптимизировать ({best_model['Time (seconds)']:.2f} сек)")
    print("   ⚠ Рекомендуется использовать для стабильных данных или уменьшить сложность")

print(f"\n   🎯 ИТОГОВАЯ РЕКОМЕНДАЦИЯ:")
print(f"   Для продакшена рекомендуется использовать: {best_model['Pipeline']}")
print(f"   Эта модель обеспечивает баланс между:")
print(f"     • Точностью: RMSE = {best_model['RMSE']:.4f}, R² = {best_model['R2 Score']:.4f}")
print(f"     • Скоростью обучения: {best_model['Time (seconds)']:.2f} секунд")
print(f"     • Стабильностью: MAE = {best_model['MAE']:.4f}")

# Выводы о принципах AutoML
print(f"\n🤖 ПРИМЕНЕНИЕ ПРИНЦИПОВ AUTOML:")
if 'Ensemble' in best_model['Pipeline'] or 'ансамбль' in best_model['Pipeline']:
    print("   ✓ Принцип ансамблирования оказался наиболее эффективным")
    print("   ✓ Комбинация нескольких моделей дала лучший результат")
elif 'Optimized' in best_model['Pipeline'] or 'Optuna' in best_model['Pipeline']:
    print("   ✓ Принцип автоматической оптимизации гиперпараметров показал эффективность")
    print("   ✓ Систематический поиск оптимальных параметров улучшил качество")
elif 'LAMA' in best_model['Pipeline']:
    print("   ✓ Полноценный AutoML фреймворк показал хорошие результаты")
    print("   ✓ Автоматизация всех этапов ML pipeline эффективна")
else:
    print("   ✓ Автоматическая предобработка и выбор признаков улучшили качество")
    print("   ✓ Применение принципов AutoML дало конкурентные результаты")

print("\n" + "="*80)


## Заключение

В данном проекте были:

1. **Проведен анализ целевой переменной**: изучено распределение рейтинга, выявлены аномальные значения, проведен временной анализ.

2. **Проведен анализ признаков**: классифицированы признаки по типам, проанализированы пропущенные значения, изучены зависимости между признаками, определена важность признаков.

3. **Создан бейзлайн с LightAutoML**: протестированы 2 различные конфигурации, выбрана лучшая.

4. **Реализовано собственное решение**: созданы 3 различных пайплайна с разными подходами к предобработке и моделированию.

5. **Проведено сравнение результатов**: все модели сравнены по метрикам RMSE, MAE и R2 Score.

Лучшая модель будет использована для генерации финальных предсказаний на тестовом наборе данных.
